# ALPHA
Atenção, como este modelo usa (moving averages/somas de últimos x jogos), nunca usamos os primeiros jogos (que ainda não têm médias) para testar/treinar o modelo. 

Vamos fazer um modelo para cada grupo de odds

In [1]:
import pandas as pd
import xlwings as xw
import openpyxl as op
import numpy as np
import datetime as dt
import numpy as np
import os
import math
import random
import pickle

In [2]:
pd.set_option('display.max_rows',2000)
pd.set_option('display.max_columns',1000)

In [3]:
from Multi_Ada_Boost1 import *

Amount_of_Say, Weights, ... vão estar na ordem das unidades com 6 casas decimais
também está feito apenas para variáveis 0/1


# Funções de tratamento de dados

`odd_to_file` dá-nos a folha do ficheiro com a Odd que queremos (apenas uma odd, NÃO  intervalos de Odds)

In [5]:
def odd_to_file(path,odd):
    o=round(odd*100)
    ind=int((o-100)//10)+1
    file=pd.read_excel(path+'.xlsx',ind)
    return file

`prepare_df` prepara a DataFame para se usar no modelo. (Basicamente, tira as colunas que não se usam no modelo)

In [5]:
def prepare_df(file):
    file=file.drop(['Div','Date','LO_Team','HO_Team','LowOdd'],axis=1)
    return file

`train_test_split` é a minha versão do módulo sklearn. Parte fixe é que não precisamos de separar o X e o Y :D 

In [6]:
def train_test_split(file,test_percentage=10):
    size=len(file.index)
    print(size,'size')
    test_size=int((size*test_percentage)/100)
    print(test_size,'test_size')
    limit=size-test_size
    print(limit)
    test_ind=random.randrange(0,limit)
    print(test_ind,test_ind+test_size)
    Test=file[test_ind:(test_ind+test_size)].reset_index(drop=True)
    Train=pd.concat([file[:test_ind],file[(test_ind+test_size):]],sort=False).reset_index(drop=True)
    return Train, Test

`round_odds` arredonda as Draw/High_Odds dependendo da Odd analisada

`round_type1` arredonda entre "inteiro/0.5"

In [7]:
def round_type1(odd):
    i=int(odd)
    Y=odd-i
    if Y>=0.75:
        return (i+1)
    elif Y<0.25:
        return (i)
    else:
        return (i+0.5)

In [8]:
def round_odd(file,odd):
    size=len(file.index)
    new_draw=[]
    new_high=[]
    if 1<=odd<=1.3:
         for i in range(size):
                new_draw+=[round(file.iloc[i]['DrawOdd'])]
                new_high+=[round(file.iloc[i]['HighOdd'])]
    elif 1.4<=odd<=1.6:
        for i in range(size):
            new_draw+=[round_type1(file.iloc[i]['DrawOdd'])]
            new_high+=[round(file.iloc[i]['HighOdd'])]
    else:
        for i in range(size):
            new_draw+=[round_type1(file.iloc[i]['DrawOdd'])]
            new_high+=[round_type1(file.iloc[i]['HighOdd'])]
        
    file['DrawOdd']=new_draw
    file['HighOdd']=new_high
    return file

In [9]:
df14=odd_to_file('BigAlpha18_20',1.4)
df14=prepare_df(df14)
df14=round_odd(df14,1.4)
df14_Train, df14_Test=train_test_split(df14,test_percentage=10)

357 size
35 test_size
322
236 271


In [9]:
t=[]
f=[]
for i in range(len(train.index)):
    l=pd.DataFrame(train.iloc[i,:]).T
    if q.match(l):
        t+=[l]
    else:
        f+=[l]
df_T=pd.concat(t,sort=False).reset_index(drop=True)
df_F=pd.concat(f,sort=False).reset_index(drop=True)

In [10]:
t=[]
f=[]
for i in range(len(test.index)):
    l=pd.DataFrame(test.iloc[i,:]).T
    if q.match(l):
        t+=[l]
    else:
        f+=[l]
test_T=pd.concat(t,sort=False).reset_index(drop=True)
test_F=pd.concat(f,sort=False).reset_index(drop=True)

In [11]:
df_T2=pd.concat([df_T['LG_Stats_Diff'],df_T['Won/NotWon']],axis=1,sort=False)
df_F2=pd.concat([df_F['LG_Stats_Diff'],df_F['Won/NotWon']],axis=1,sort=False)

In [12]:
test_T2=pd.concat([test_T['LG_Stats_Diff'],test_T['Won/NotWon']],axis=1,sort=False)
test_F2=pd.concat([test_F['LG_Stats_Diff'],test_F['Won/NotWon']],axis=1,sort=False)

In [13]:
zero_counter=0
one_counter=0
for i in range(len(df_T2.index)):
    if df_T2.iloc[i][-1]==0:
        zero_counter+=1
    else:
        one_counter+=1
        
print('0:',zero_counter,'1:',one_counter)

0: 144 1: 154


In [14]:
zero_counter=0
one_counter=0
for i in range(len(test_T2.index)):
    if test_T2.iloc[i][-1]==0:
        zero_counter+=1
    else:
        one_counter+=1
        
print('0:',zero_counter,'1:',one_counter)

0: 16 1: 19


In [15]:
zero_counter=0
one_counter=0
for i in range(len(df_F2.index)):
    if df_F2.iloc[i][-1]==0:
        zero_counter+=1
    else:
        one_counter+=1
        
print('0:',zero_counter,'1:',one_counter)

0: 9 1: 32


In [16]:
zero_counter=0
one_counter=0
for i in range(len(test_F2.index)):
    if test_F2.iloc[i][-1]==0:
        zero_counter+=1
    else:
        one_counter+=1
        
print('0:',zero_counter,'1:',one_counter)

0: 1 1: 1


In [18]:
test_T

,DrawOdd,HighOdd,Points_Diff5,LG_Stats_Diff,L_LowestLost,S_Diff5,ST_Diff5,Scrd_Diff5,Concd_Diff5,WinCounter_L,WinCounter_H,ScrdCounter_L,ScrdCounter_H,ConcdCounter_L,ConcdCounter_H,LeagueOdd_Counter,Won/NotWon
0,4.0,4.5,0.0,-2.0,1.5,-2.0,-1.0,0.0,0.0,-4.0,-3.0,-1.0,1.0,-6.0,-6.0,2.0,1.0
1,3.5,5.0,9.0,3.0,2.3,4.0,2.0,0.0,-2.0,2.0,-4.0,6.0,-2.0,2.0,-7.0,-1.0,0.0
2,4.0,5.0,1.0,1.0,1.5,4.0,-1.0,0.0,-1.0,1.0,1.0,2.0,5.0,1.0,-4.0,3.0,0.0
3,3.5,5.0,3.0,4.0,2.1,-1.0,2.0,0.0,0.0,1.0,-9.0,1.0,-1.0,1.0,-6.0,-2.0,1.0
4,4.0,4.5,11.0,5.0,3.8,1.0,1.0,2.0,0.0,7.0,-4.0,8.0,-2.0,-2.0,-3.0,1.0,0.0
5,4.0,5.0,-1.0,-2.0,1.5,-5.0,0.0,0.0,1.0,-1.0,3.0,6.0,3.0,-5.0,1.0,2.0,1.0
6,4.0,4.5,1.0,0.0,1.2,8.0,3.0,2.0,-1.0,1.0,4.0,1.0,9.0,5.0,3.0,3.0,1.0
7,3.5,5.0,-2.0,-2.0,2.0,4.0,1.0,-1.0,-1.0,-3.0,-3.0,1.0,-2.0,-4.0,-4.0,4.0,1.0
8,3.5,4.5,9.0,2.0,1.4,1.0,1.0,1.0,-1.0,1.0,-6.0,2.0,1.0,1.0,-6.0,5.0,0.0
9,3.5,5.0,7.0,4.0,1.5,2.0,1.0,1.0,-1.0,-2.0,-5.0,1.0,1.0,-2.0,-5.0,-1.0,0.0


In [5]:
train2=first_weights(train,auto=False,var=0,var_percentage=63)
q,b,Err=Ada_best_stump(train2)

0 coluna
já foste
[3.25, 3.75] values
0 tipo de pergunta
0.6298608475648317 True ->1, False->0
0.3701391524351685 True ->0, False->1
--------------------------------------------------------------
0.4158096141682496 True ->1, False->0
0.5841903858317512 True ->0, False->1
--------------------------------------------------------------
1 tipo de pergunta
0.3701391524351685 --------Le Mistake du Colun-------
1 coluna
já foste
[4.25, 4.75, 5.25] values
0 tipo de pergunta
0.6258823529411759 True ->1, False->0
0.37411764705882444 True ->0, False->1
--------------------------------------------------------------
0.5374067046173312 True ->1, False->0
0.46259329538266764 True ->0, False->1
--------------------------------------------------------------
0.3848987982289701 True ->1, False->0
0.6151012017710303 True ->0, False->1
--------------------------------------------------------------
1 tipo de pergunta
0.37411764705882444 --------Le Mistake du Colun-------
2 coluna
já foste
[-8.5, -7.5, -6.5,

0.6000632511068928 True ->1, False->0
0.3999367488931058 True ->0, False->1
--------------------------------------------------------------
0.6129728020240344 True ->1, False->0
0.38702719797596513 True ->0, False->1
--------------------------------------------------------------
0.6213472485768492 True ->1, False->0
0.3786527514231506 True ->0, False->1
--------------------------------------------------------------
0.629721695129664 True ->1, False->0
0.37027830487033603 True ->0, False->1
--------------------------------------------------------------
0.6359677419354832 True ->1, False->0
0.364032258064517 True ->0, False->1
--------------------------------------------------------------
0.3685673624288434 True ->1, False->0
0.631432637571157 True ->0, False->1
--------------------------------------------------------------
0.35889943074003877 True ->1, False->0
0.6411005692599616 True ->0, False->1
--------------------------------------------------------------
0.3614452877925372 True ->1

0.6183839342188473 True ->1, False->0
0.38161606578115137 True ->0, False->1
--------------------------------------------------------------
0.6267583807716621 True ->1, False->0
0.3732416192283368 True ->0, False->1
--------------------------------------------------------------
0.635132827324477 True ->1, False->0
0.36486717267552227 True ->0, False->1
--------------------------------------------------------------
0.6413788741302962 True ->1, False->0
0.35862112586970324 True ->0, False->1
--------------------------------------------------------------
0.4365370018975351 True ->1, False->0
0.5634629981024658 True ->0, False->1
--------------------------------------------------------------
0.47313725490196334 True ->1, False->0
0.5268627450980378 True ->0, False->1
--------------------------------------------------------------
0.5198228969006973 True ->1, False->0
0.48017710309930267 True ->0, False->1
--------------------------------------------------------------
0.5578557874762805 True

0.49497786211258926 True ->1, False->0
0.5050221378874113 True ->0, False->1
--------------------------------------------------------------
0.5012239089184083 True ->1, False->0
0.4987760910815922 True ->0, False->1
--------------------------------------------------------------
0.4060025300442773 True ->1, False->0
0.5939974699557233 True ->0, False->1
--------------------------------------------------------------
0.42728652751423335 True ->1, False->0
0.5727134724857674 True ->0, False->1
--------------------------------------------------------------
0.44019607843137465 True ->1, False->0
0.5598039215686262 True ->0, False->1
--------------------------------------------------------------
0.4485705249841894 True ->1, False->0
0.5514294750158115 True ->0, False->1
--------------------------------------------------------------
0.45694497153700425 True ->1, False->0
0.5430550284629968 True ->0, False->1
--------------------------------------------------------------
0.46319101834282345 Tru

0.3661606578115126 True ->1, False->0
0.6338393421884878 True ->0, False->1
--------------------------------------------------------------
0.37439595192915975 True ->1, False->0
0.6256040480708407 True ->0, False->1
--------------------------------------------------------------
0.3785135989879833 True ->1, False->0
0.6214864010120171 True ->0, False->1
--------------------------------------------------------------
0.37666350411132293 True ->1, False->0
0.6233364958886775 True ->0, False->1
--------------------------------------------------------------
0.370695762175839 True ->1, False->0
0.6293042378241614 True ->0, False->1
--------------------------------------------------------------
0.36884566729917867 True ->1, False->0
0.6311543327008218 True ->0, False->1
--------------------------------------------------------------
0.37920936116382137 True ->1, False->0
0.620790638836179 True ->0, False->1
--------------------------------------------------------------
0.3733807716635051 True -

...

0.3489531941808989 True ->1, False->0
0.6510468058191015 True ->0, False->1
--------------------------------------------------------------
0.3594560404807092 True ->1, False->0
0.6405439595192912 True ->0, False->1
--------------------------------------------------------------
0.3700980392156872 True ->1, False->0
0.6299019607843133 True ->0, False->1
--------------------------------------------------------------
0.3952213788741316 True ->1, False->0
0.604778621125869 True ->0, False->1
--------------------------------------------------------------
0.41423782416192445 True ->1, False->0
0.5857621758380762 True ->0, False->1
--------------------------------------------------------------
0.438067678684379 True ->1, False->0
0.5619323213156219 True ->0, False->1
--------------------------------------------------------------
0.48405755850727655 True ->1, False->0
0.5159424414927246 True ->0, False->1
--------------------------------------------------------------
0.5085831752055674 True ->1

0.37125237191650945 True ->1, False->0
0.6287476280834909 True ->0, False->1
--------------------------------------------------------------
0.38189437065148746 True ->1, False->0
0.618105629348513 True ->0, False->1
--------------------------------------------------------------
0.40701771030993183 True ->1, False->0
0.5929822896900688 True ->0, False->1
--------------------------------------------------------------
0.4260341555977247 True ->1, False->0
0.573965844402276 True ->0, False->1
--------------------------------------------------------------
0.4498640101201793 True ->1, False->0
0.5501359898798217 True ->0, False->1
--------------------------------------------------------------
0.4958538899430769 True ->1, False->0
0.5041461100569244 True ->0, False->1
--------------------------------------------------------------
0.5203795066413675 True ->1, False->0
0.47962049335863216 True ->0, False->1
--------------------------------------------------------------
0.5580929791271333 True -

0.641976597090447 True ->1, False->0
0.3580234029095507 True ->0, False->1
--------------------------------------------------------------
0.6460942441492705 True ->1, False->0
0.3539057558507272 True ->0, False->1
--------------------------------------------------------------
0.43796963946869255 True ->1, False->0
0.5620303605313084 True ->0, False->1
--------------------------------------------------------------
0.4839595192915902 True ->1, False->0
0.516040480708411 True ->0, False->1
--------------------------------------------------------------
0.5084851359898822 True ->1, False->0
0.4915148640101186 True ->0, False->1
--------------------------------------------------------------
0.546198608475648 True ->1, False->0
0.4538013915243501 True ->0, False->1
--------------------------------------------------------------
0.6143074003795037 True ->1, False->0
0.3856925996204925 True ->0, False->1
--------------------------------------------------------------
0.6313345983554688 True ->1, 

0.6278716002530038 True ->1, False->0
0.37212839974699646 True ->0, False->1
--------------------------------------------------------------
0.44033523086654225 True ->1, False->0
0.5596647691334586 True ->0, False->1
--------------------------------------------------------------
0.4970651486401024 True ->1, False->0
0.5029348513598969 True ->0, False->1
--------------------------------------------------------------
0.5484250474383288 True ->1, False->0
0.4515749525616686 True ->0, False->1
--------------------------------------------------------------
0.5704048070841217 True ->1, False->0
0.4295951929158756 True ->0, False->1
--------------------------------------------------------------
0.5999240986717251 True ->1, False->0
0.40007590132827336 True ->0, False->1
--------------------------------------------------------------
0.612694497153699 True ->1, False->0
0.3873055028463002 True ->0, False->1
--------------------------------------------------------------
0.6231973434535094 True -

0.41682479443390413 True ->1, False->0
0.5831752055660966 True ->0, False->1
--------------------------------------------------------------
0.4230708412397233 True ->1, False->0
0.5769291587602774 True ->0, False->1
--------------------------------------------------------------
0.42519924098671896 True ->1, False->0
0.5748007590132819 True ->0, False->1
--------------------------------------------------------------
0.423209993674891 True ->1, False->0
0.5767900063251098 True ->0, False->1
--------------------------------------------------------------
0.421220746363063 True ->1, False->0
0.5787792536369378 True ->0, False->1
--------------------------------------------------------------
0.38730550284630094 True ->1, False->0
0.6126944971536995 True ->0, False->1
--------------------------------------------------------------
0.39355154965212014 True ->1, False->0
0.6064484503478804 True ->0, False->1
--------------------------------------------------------------
0.3956799493991157 True -

In [8]:
train.columns[q.column],q.value1,q.value2,b

('ConcdCounter_H', -8.5, 3.5, False)

In [4]:
red=open('AB_ALPHA_17.pickle','rb')
Ada=pickle.load(red)
train=pickle.load(red)
test=pickle.load(red)
red.close()

In [9]:
train

,DrawOdd,HighOdd,Points_Diff5,LG_Stats_Diff,L_LowestLost,S_Diff5,ST_Diff5,Scrd_Diff5,Concd_Diff5,WinCounter_L,WinCounter_H,ScrdCounter_L,ScrdCounter_H,ConcdCounter_L,ConcdCounter_H,LeagueOdd_Counter,Won/NotWon
0,3.5,4.5,4,8,2.3,0,1,-1,-1,1,-5,1,4,-2,-6,-1,1
1,3.5,4.5,5,7,2.3,0,-1,1,0,-1,-4,2,1,-3,-4,1,0
2,3.5,4.5,3,-1,1.8,-1,-3,0,0,-2,1,4,1,-2,-8,-1,1
3,3.0,4.5,3,0,1.2,-1,-1,0,0,-1,-2,4,3,-4,-7,1,0
4,3.0,4.5,-5,-9,1.5,1,2,0,1,1,6,5,9,-2,4,-1,1
5,3.5,4.5,3,1,1.2,4,3,1,0,2,-1,7,2,-1,-1,1,1
6,3.0,4.5,-2,0,1.6,-4,-3,1,1,-1,1,-1,3,-3,1,2,1
7,3.5,4.5,11,6,2.0,7,5,1,-2,8,-4,8,-2,8,-4,3,1
8,3.5,4.5,7,3,1.2,3,2,1,-1,-1,-4,-1,-3,1,-1,4,0
9,3.5,4.5,5,6,2.0,-5,-3,0,0,1,-14,2,2,1,-3,-1,0


In [16]:
pd.concat([train['LG_Stats_Diff'],train['Won/NotWon']],axis=1,sort=False)

,LG_Stats_Diff,Won/NotWon
0,8,1
1,7,0
2,-1,1
3,0,0
4,-9,1
5,1,1
6,0,1
7,6,1
8,3,0
9,6,0


In [10]:
used_columns(Ada,names=True,df=test)

{'DrawOdd': 0,
 'HighOdd': 0,
 'Points_Diff5': 3,
 'LG_Stats_Diff': 7,
 'L_LowestLost': 10,
 'S_Diff5': 2,
 'ST_Diff5': 2,
 'Scrd_Diff5': 2,
 'Concd_Diff5': 1,
 'WinCounter_L': 4,
 'WinCounter_H': 6,
 'ScrdCounter_L': 6,
 'ScrdCounter_H': 4,
 'ConcdCounter_L': 5,
 'ConcdCounter_H': 7,
 'LeagueOdd_Counter': 1,
 'Won/NotWon': 0}

In [11]:
Q_info(Ada,names=True,df=test)

[[0, 0.163645, 'LG_Stats_Diff', 1, 2.5, 7.5],
 [1, 0.137042, 'L_LowestLost', 1, 1.35, 2.55],
 [2, 0.143136, 'ScrdCounter_L', 1, 2.5, 8.5],
 [3, 0.12504, 'ConcdCounter_L', 1, -6.5, 0.0],
 [4, 0.128859, 'ConcdCounter_H', 1, -8.5, -2.5],
 [5, 0.136811, 'ConcdCounter_H', 1, -3.5, 3.5],
 [6, 0.10059, 'S_Diff5', 1, 2.5, 10.5],
 [7, 0.104003, 'Points_Diff5', 1, -1.5, 3.5],
 [8, 0.096781, 'ScrdCounter_H', 1, 1.5, 6.5],
 [9, 0.094282, 'Scrd_Diff5', 0, 0.5, None],
 [10, 0.088464, 'ConcdCounter_H', 1, -8.5, -2.5],
 [11, 0.095762, 'WinCounter_L', 1, -4.5, -1.5],
 [12, 0.096916, 'ScrdCounter_L', 1, 0.0, 8.5],
 [13, 0.081236, 'L_LowestLost', 1, 1.15, 1.95],
 [14, 0.100871, 'WinCounter_H', 1, -6.5, -3.5],
 [15, 0.085425, 'L_LowestLost', 0, 1.45, None],
 [16, 0.076704, 'Points_Diff5', 1, -2.5, 12.5],
 [17, 0.103342, 'ConcdCounter_H', 1, 0.0, 3.5],
 [18, 0.07278, 'LG_Stats_Diff', 1, 0.5, 4.5],
 [19, 0.075863, 'WinCounter_L', 1, 2.5, 5.5],
 [20, 0.089611, 'L_LowestLost', 0, 1.15, None],
 [21, 0.082509, 

In [15]:
Ada_cmat(train,Ada,auto=True,var=1,limite=60)

predicted [0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,

(array([[146,  56],
        [ 40,  97]]), 72.28)

In [94]:
df14_Test.columns

Index(['DrawOdd', 'HighOdd', 'Points_Diff5', 'LG_Stats_Diff', 'L_LowestLost',
       'S_Diff5', 'ST_Diff5', 'Scrd_Diff5', 'Concd_Diff5', 'WinCounter_L',
       'WinCounter_H', 'ScrdCounter_L', 'ScrdCounter_H', 'ConcdCounter_L',
       'ConcdCounter_H', 'LeagueOdd_Counter', 'Won/NotWon'],
      dtype='object')

In [127]:
pd.concat([df14_Test['Points_Diff5'],df14_Test['Won/NotWon']],sort=False,axis=1)

,Points_Diff5,Won/NotWon
0,0,1
1,5,0
2,9,1
3,-2,1
4,9,0
5,4,0
6,5,0
7,6,0
8,1,1
9,1,0


In [98]:
row=pd.DataFrame({'ok':[5,7,1,10,1.5,3,2,1,-2,3,-1,-1,3,1,-3,-1]}).T
Ada_predict_rowx(row,Ada,auto=True)

1 2.0754960000000002
0 2.4927970000000004


0

In [25]:
df23=odd_to_file('BigAlpha18_20',2.3)
df23=prepare_df(df23)
df23=round_odd(df23,2)
df23_Train, df23_Test=train_test_split(df23,test_percentage=10)

351 size
35 test_size
316
113 148


In [131]:
Ada14_first=Ada

In [132]:
Ada14_first

In [119]:
def Ada_predict_rowx(row,adaforest,auto=True,var=1,limite=70):
    size=adaforest.size()
    ones=0 #guardando o a_say das previsões de 1
    zeros=0  #guardando o a_say das previsões de 1
    for i in range(size):
        stump=adaforest.get_stump(i)
        q=stump.question
        if q.match(row):
            if stump.true_branch.var==1:
                ones+=stump.a_of_say
            else:
                zeros+=stump.a_of_say
        else:
            if stump.false_branch.var==1:
                ones+=stump.a_of_say
            else:
                zeros+=stump.a_of_say
    #print('1',ones)
    #print('0',zeros)
    if auto:
        if ones>zeros:
            return 1
        else:
            return 0
    else:
        if var==1:
            if round(100*(ones/(zeros+ones)),2)>=limite:
                return 1
            else:
                return 0
        elif var==0:
            if round(100*(zeros/(zeros+ones)),2)>=limite:
                return 0
            else:
                return 1
            
            
def Ada_predictx(df,adaforest,auto=True,var=1,limite=70):
    size1=len(df.index)
    predictions=[]
    for i in range(size1):
        row=pd.DataFrame(df.iloc[i,:]).T
        pred=Ada_predict_rowx(row,adaforest,auto=auto,var=var,limite=limite)
        predictions+=[pred]
    return predictions


def Ada_cmatx(df,adaforest,auto=False,var=1,limite=60):
    real=list(df.iloc[:,-1]) #VERIFICAR, mas como adicionamos as colunas dos pesos nas funções, as df original fica igual
    predicted=Ada_predictx(df,adaforest,auto=auto,var=var,limite=limite)
    print('predicted',predicted)
    cmat=np.array([[0,0],[0,0]])
    for i in range(len(real)):
        if real[i]==1 and predicted[i]==1:
            cmat[0][0]+=1
        elif real[i]==1 and predicted[i]==0:
            cmat[1][0]+=1
        elif real[i]==0 and predicted[i]==1:
            cmat[0][1]+=1
        elif real[i]==0 and predicted[i]==0:
            cmat[1][1]+=1
        else:
            print('Erro na cmat_forest')
            return 'Erro na cmat_forest'
    return cmat, pred_percent(cmat,1-var)

def Q_info(adaforest,names=False,df=None):
    #devolve os a_of_say,tipo, coluna e valores de cada pergunta (com o seu índice)
    size= adaforest.size()
    info=[]
    for i in range(size):
        stump=adaforest.get_stump(i)
        if names: #dá nos o nome das colunas em vez do índice
            coluna=df.columns[stump.question.column]
        else:
            coluna=stump.question.column
        info+=[[i,stump.a_of_say,coluna,stump.question.tipo,stump.question.value1,stump.question.value2]]
    return info

def used_columns(adaforest,names=False,df=None):
    #dá um dicionário com as vezes que cada coluna foi usada
    dic={}
    if names: #se tivermos df, temos todas as colunas no dic
        for c in df.columns:
            dic[c]=0
    size= adaforest.size()
    for i in range(size):
        stump=adaforest.get_stump(i)
        if names: #dá nos o nome das colunas em vez do índice
            coluna=df.columns[stump.question.column]
        else:
            coluna=stump.question.column
        if coluna not in dic:
            dic[coluna]=0
        dic[coluna]+=1
    return dic

# Guardar Modelos
### Pickle

Há um ficheiro `ZZPickle.txt` nas Apostas que diz os `.pickle` files que existem e o que têm

In [5]:
import pickle

In [1]:
def save_to_pickle(var,file): #se len(var)>1, var tem de ser uma lista
    with open(file,'wb') as f:
        pickle.dump(var,f)

In [9]:
def load_from_pickle(var,file): #esta função não dá para correr mas (só se mudarmos umas coisinhas)
    #da para fazer load fora da função 
    #Por isso fica aqui para ver como se faz
    with open(file,'rb') as f:
        var=pickle.load(f)

In [133]:
save_to_pickle(Ada14_first,'ALPHA.pickle')

In [3]:
load_from_pickle(Ada14_first,'ALPHA.pickle')

NameError: name 'Ada14_first' is not defined

In [7]:
with open('ALPHA.pickle','rb') as f:
    Ada14_first=pickle.load(f)

In [8]:
Ada14_first